In [1]:
import numpy as np
import pandas as pd

In [2]:
DSX = pd.read_csv("DrugSpaceX-S-sample-smiles.smi", sep="\t")

In [3]:
DSX = DSX.dropna()

In [4]:
DSX

,SMILES,DE ID,LogP,MW,H_A,H_D,R_B,TPAS,QED,SAScore,Parents ID,NS ID,PDB_id,DrugBank_id,BindingDB_id,ChEMBL_id,CCDC_id,PAINS,SureCHEMBL ID,Novelty
0,O=Cc1ncc(COP(=O)(O)O)c(C=O)c1O,DE2406,0.02,261.00,8,3,5,134.02,0.5000,3.35,DE10,NS4,ND,ND,ND,ND,ND,False,ND,1.0
1,CCc1ncc(COP(=O)(O)O)c(C=O)c1O,DE2407,0.77,261.04,7,3,5,116.95,0.5300,3.13,DE10,NS15,ND,ND,ND,ND,ND,False,ND,1.0
2,Cc1ncc(COP(=O)(O)O)c(N=O)c1O,DE2408,1.10,248.02,8,3,4,129.31,0.5400,3.31,DE10,NS17,ND,ND,ND,ND,ND,False,ND,1.0
3,Cc1ncc(CCP(=O)(O)O)c(C=O)c1O,DE2409,0.63,245.05,6,3,4,107.72,0.5300,3.11,DE10,NS40,ND,ND,50336044,CHEMBL3427368,ND,False,ND,1.0
4,Cc1ncc(C(O)OP(=O)(O)O)c(C=O)c1O,DE2410,0.01,263.02,8,4,4,137.18,0.3400,3.78,DE10,NS55,ND,ND,ND,ND,ND,False,ND,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937225,CNC[C@@H]1CC[C@@H](N)[C@@H](O[C@@H]2C3C(C[C@@H...,DE953121,-3.47,489.32,12,11,7,199.73,0.1759,5.62,DE2396,NS18754,ND,ND,ND,ND,ND,False,ND,1.0
937226,CNC[C@@H]1CC[C@@H](N)[C@@H](O[C@@H]2[C@@H](N)C...,DE953122,-3.47,489.32,12,11,7,199.73,0.1759,5.62,DE2396,NS18754,ND,ND,ND,ND,ND,False,ND,1.0
937227,CNC[C@@H]1CC[C@@H](N)[C@@H](O[C@@H]2C3C([C@H]2...,DE953123,-3.47,489.32,12,11,7,199.73,0.1759,5.59,DE2396,NS18754,ND,ND,ND,ND,ND,False,ND,1.0
937228,CNC[C@@H]1CC[C@@H](N)[C@@H](O[C@@H]2[C@@H](N)C...,DE953124,-3.47,489.32,12,11,7,199.73,0.1759,5.61,DE2396,NS18754,ND,ND,ND,ND,ND,False,ND,1.0


In [5]:
drugs = pd.read_csv("drugs_clear.csv", index_col=0)

In [6]:
drugs

,generic_name,smiles
0,Abacavir,NC1=NC2=C(N=CN2[C@@H]2C[C@H](CO)C=C2)C(NC2CC2)=N1
1,Abiraterone,CC(=O)O[C@H]1CC[C@]2(C)C3CC[C@@]4(C)C(CC=C4C4=...
2,Acamprosate,CC(=O)NCCCS(O)(=O)=O
3,Acarbose,C[C@H]1O[C@H](O[C@@H]2[C@@H](CO)O[C@H](O[C@@H]...
4,Acebutolol,CCCC(=O)NC1=CC(C(C)=O)=C(OCC(O)CNC(C)C)C=C1
...,...,...
1492,Zolmitriptan,CN(C)CCC1=CNC2=CC=C(C[C@H]3COC(=O)N3)C=C12
1493,Zolpidem,CN(C)C(=O)CC1=C(N=C2C=CC(C)=CN12)C1=CC=C(C)C=C1
1494,Zonisamide,NS(=O)(=O)CC1=NOC2=CC=CC=C12
1495,Zopiclone,CN1CCN(CC1)C(=O)OC1N(C(=O)C2=NC=CN=C12)C1=NC=C...


In [7]:
smiles_drugs = drugs["smiles"]
smiles_DSX = DSX["SMILES"]

In [8]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs

In [9]:
#creo la funzione per comparare gli smiles
def tanimoto_calc(smi1, smi2):
    mol1 = Chem.MolFromSmiles(smi1)
    mol2 = Chem.MolFromSmiles(smi2)
    fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, 3, nBits=2048)
    fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, 3, nBits=2048)
    s = round(DataStructs.TanimotoSimilarity(fp1,fp2),3)
    return s

In [12]:
#Creo un insieme vuoto in cui verranno inseriti gli smiles di DrugSpaceX simili agli smiles dei farmaci
similar_smiles = set()

In [17]:
for drugsmiles in smiles_drugs:
    for dsxsmiles in smiles_DSX:
        if tanimoto_calc(drugsmiles, dsxsmiles) >= 0.8:
            similar_smiles.add(dsxsmiles)

[17:33:52] WARNING: not removing hydrogen atom without neighbors
[17:34:31] WARNING: not removing hydrogen atom without neighbors
[17:40:35] WARNING: not removing hydrogen atom without neighbors
[17:43:39] WARNING: not removing hydrogen atom without neighbors
[17:43:42] Conflicting single bond directions around double bond at index 2.
[17:43:42]   BondStereo set to STEREONONE and single bond directions set to NONE.
[17:43:42] Conflicting single bond directions around double bond at index 1.
[17:43:42]   BondStereo set to STEREONONE and single bond directions set to NONE.
[17:43:42] Conflicting single bond directions around double bond at index 1.
[17:43:42]   BondStereo set to STEREONONE and single bond directions set to NONE.
[17:43:42] Conflicting single bond directions around double bond at index 1.
[17:43:42]   BondStereo set to STEREONONE and single bond directions set to NONE.
[17:43:43] Conflicting single bond directions around double bond at index 3.
[17:43:43]   BondStereo set

[19:14:15] Conflicting single bond directions around double bond at index 2.
[19:14:15]   BondStereo set to STEREONONE and single bond directions set to NONE.
[19:14:17] Conflicting single bond directions around double bond at index 3.
[19:14:17]   BondStereo set to STEREONONE and single bond directions set to NONE.
[19:14:17] Conflicting single bond directions around double bond at index 2.
[19:14:17]   BondStereo set to STEREONONE and single bond directions set to NONE.
[19:17:55] WARNING: not removing hydrogen atom without neighbors
[19:24:48] WARNING: not removing hydrogen atom without neighbors
[19:25:14] WARNING: not removing hydrogen atom without neighbors
[19:29:25] WARNING: not removing hydrogen atom without neighbors
[19:31:31] WARNING: not removing hydrogen atom without neighbors
[19:31:34] Conflicting single bond directions around double bond at index 2.
[19:31:34]   BondStereo set to STEREONONE and single bond directions set to NONE.
[19:31:34] Conflicting single bond dire

[21:13:18] Conflicting single bond directions around double bond at index 3.
[21:13:18]   BondStereo set to STEREONONE and single bond directions set to NONE.
[21:13:18] Conflicting single bond directions around double bond at index 2.
[21:13:18]   BondStereo set to STEREONONE and single bond directions set to NONE.
[21:16:40] Conflicting single bond directions around double bond at index 2.
[21:16:40]   BondStereo set to STEREONONE and single bond directions set to NONE.
[21:16:42] Conflicting single bond directions around double bond at index 3.
[21:16:42]   BondStereo set to STEREONONE and single bond directions set to NONE.
[21:16:42] Conflicting single bond directions around double bond at index 2.
[21:16:42]   BondStereo set to STEREONONE and single bond directions set to NONE.
[21:21:55] WARNING: not removing hydrogen atom without neighbors
[21:29:45] WARNING: not removing hydrogen atom without neighbors
[21:30:15] WARNING: not removing hydrogen atom without neighbors
[21:36:12]

[23:30:24] Conflicting single bond directions around double bond at index 3.
[23:30:24]   BondStereo set to STEREONONE and single bond directions set to NONE.
[23:30:24] Conflicting single bond directions around double bond at index 2.
[23:30:24]   BondStereo set to STEREONONE and single bond directions set to NONE.
[23:33:45] Conflicting single bond directions around double bond at index 2.
[23:33:45]   BondStereo set to STEREONONE and single bond directions set to NONE.
[23:33:47] Conflicting single bond directions around double bond at index 3.
[23:33:47]   BondStereo set to STEREONONE and single bond directions set to NONE.
[23:33:47] Conflicting single bond directions around double bond at index 2.
[23:33:47]   BondStereo set to STEREONONE and single bond directions set to NONE.
[23:38:51] WARNING: not removing hydrogen atom without neighbors
[23:53:57] WARNING: not removing hydrogen atom without neighbors
[23:54:58] WARNING: not removing hydrogen atom without neighbors
[00:02:19]

[02:08:10] Conflicting single bond directions around double bond at index 3.
[02:08:10]   BondStereo set to STEREONONE and single bond directions set to NONE.
[02:08:10] Conflicting single bond directions around double bond at index 2.
[02:08:10]   BondStereo set to STEREONONE and single bond directions set to NONE.
[02:10:27] Conflicting single bond directions around double bond at index 2.
[02:10:27]   BondStereo set to STEREONONE and single bond directions set to NONE.
[02:10:29] Conflicting single bond directions around double bond at index 3.
[02:10:29]   BondStereo set to STEREONONE and single bond directions set to NONE.
[02:10:29] Conflicting single bond directions around double bond at index 2.
[02:10:29]   BondStereo set to STEREONONE and single bond directions set to NONE.
[02:13:54] WARNING: not removing hydrogen atom without neighbors
[02:21:53] WARNING: not removing hydrogen atom without neighbors
[02:22:25] WARNING: not removing hydrogen atom without neighbors
[02:26:46]

KeyboardInterrupt: 

In [ ]:
similar_smiles_list = list(similar_smiles)

In [ ]:
df_similar_smiles = pd.DataFrame({"smiles": similar_smiles_list})

In [ ]:
df_similar_smiles